<div align="right"><i>Matías Torres Esteban<br>Diciembre, 2025</i></div>

# Generación Aumentada por Recuperación (RAG)

En un sistema de recuperación de información (SRI), dada una consulta $q$ de un usuario se recuperan de una colección de documentos $D$ los $k$ documentos más relevantes a $q$ [1]. El lenguaje en el que se expresan los documentos y la consulta, la forma en la que se representa en una computadora la colección $D$ y la definición de relevancia varían de acuerdo al dominio de aplicación y a las necesidades del usuario. Los motores de búsqueda de Google y Yahoo, por ejemplo, son SRIs donde los documentos son páginas web y las consultas se expresan en lenguaje natural. 

En el contexto de la inteligencia artificial y particularmente de los agentes razonadores, podemos conceptualizar a $D$ como parte de la base de conocimiento de un sistema inteligente, y a los documentos recuperados como premisas relevantes con las que se llevará a cabo un proceso de deducción para contestar $q$. Con el surgimiento de los grandes modelos de lenguaje (LLMs) se desarrollaron nuevos modelos de razonamiento subsimbólicos y neurosimbólicos que hacen uso de SRIs en el proceso de deducción [2]. Uno de estos modelos es el de **generación aumentada por recuperación** (RAG), el cual combina LLMs con SRIs para responder preguntas que requieren de conocimiento experto [3]. ChatGPT implementa este mecanismo cuando realiza búsquedas en la web:

![Ejemplo de interacción RAG con ChatGPT](imagenes/rag_messi_chatgpt.png)

En esa sección implementaremos un sistema RAG que combina LLMs con SRIs y bases de datos vectoriales para responder preguntas del dominio de psicología


In [1]:
from sentence_transformers import SentenceTransformer
from functools import partial

import pandas as pd
import numpy as np
import csv as csv
import umap.plot
import chromadb
import umap
import json

/home/matias/miniconda3/envs/neurosym/lib/python3.13/site-packages/numba/np/ufunc/dufunc.py:344: NumbaWarning: Compilation requested for previously compiled argument types ((uint32,)). This has no effect and perhaps indicates a bug in the calling code (compiling a ufunc more than once for the same signature
  warnings.warn(msg, errors.NumbaWarning)
/home/matias/miniconda3/envs/neurosym/lib/python3.13/site-packages/numba/np/ufunc/dufunc.py:344: NumbaWarning: Compilation requested for previously compiled argument types ((uint32,)). This has no effect and perhaps indicates a bug in the calling code (compiling a ufunc more than once for the same signature
  warnings.warn(msg, errors.NumbaWarning)
/home/matias/miniconda3/envs/neurosym/lib/python3.13/site-packages/numba/np/ufunc/dufunc.py:344: NumbaWarning: Compilation requested for previously compiled argument types ((uint32,)). This has no effect and perhaps indicates a bug in the calling code (compiling a ufunc more than once for the same

Importamos el LLM Encoder

In [2]:
ST =  SentenceTransformer("all-MiniLM-L6-v2")

Abrimos el lote de datos:

In [3]:
with open('./datos/psicologia.json', 'r') as f:
    chunks = json.load(f)

textos = [chunk['text'] for chunk in chunks]
print(textos[0])

Mental health is a state of mind characterized by emotional well-being, good behavioral adjustment, relative freedom from anxiety and disabling symptoms, and a capacity to establish constructive relationships and cope with the ordinary demands and stresses of life.



In [12]:
embeddings_textos = ST.encode(textos)

In [5]:
cli = chromadb.PersistentClient(path="./datos/base_semantica")

In [6]:
col = cli.create_collection(
    name="ST_med2",
    embedding_function=None,
    configuration={
        "hnsw": {
            "space": "cosine",
            "ef_construction": 1000,
            "ef_search": 200
        }
    },
    metadata = {
        "description": "Embeddings del lote de documentos de psicología"
    }
)

In [13]:
U = umap.UMAP(n_components = 2, n_neighbors = 15, min_dist = 0.1, metric = 'cosine', densmap = True)
U.fit(embeddings_textos)

UMAP(angular_rp_forest=True, densmap=True, metric='cosine', tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True})

In [14]:
hover_data = pd.DataFrame({'title': [chunk['title'] for chunk in chunks],
                           'text': [chunk['text'] for chunk in chunks]})

In [15]:
umap.plot.output_notebook()

Loading BokehJS ...

In [16]:
p = umap.plot.interactive(U, hover_data=hover_data, point_size=10)
umap.plot.show(p)

Agregamos los datos a nuestra base de datos vectorial:

In [17]:
col.add(
    ids = [chunk['title'] for chunk in chunks],
    embeddings = embeddings_textos,
    documents = textos
)

In [20]:
q = "What is melanoma?"
qe = ST.encode(q)
knn_q = col.query(query_embeddings = [qe], n_results = 5)
for d in knn_q['documents'][0]:
    print(d)

transcortical motor aphasia (TMA) is a form of aphasia in which individuals cannot initiate speech but can repeat words.
It is caused by a lesion just in front of Broca’s area.

Anedonia is the inability to enjoy experiences or activities that normally would be pleasurable.
It is one of two defining symptoms of a major depressive episode (the other being a persistent depressed mood), but it is also seen in other disorders, including schizophrenia.
It was first defined in 1897 by Théodule Ribot.

Dysthymic disorder, in DSM–IV–TR, is a mood disorder characterized by symptoms that are less severe but more enduring than those in major depressive disorder.
In DSM-5 and DSM-5-TR, the designation persistent depressive disorderis a consolidation of dysthymic disorder and chronic major depressive disorder, defined in DSM-IV.
It is also called dysthymia.

Amyloid plaque is a clump of beta-amyloid protein surrounded by degenerated dendrites that is particularly associated with symptoms of Alzheim